In [71]:
import pandas as pd
import datetime
from typing import List
import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Timeline, Grid, Bar, Map, Pie, Line

In [72]:
df = pd.read_csv('COVID-19.csv')

In [73]:
#correspond with String provided by echart
df['provinceName'] = df['provinceName'].apply(lambda y: y[:3] if y == '内蒙古自治区' or y == '黑龙江省' else y[:2])

In [74]:
#List all the province in China 加台湾
provinceName = ["云南", "内蒙古", "北京", "吉林", "四川","上海",
                "宁夏", "安徽", "山东","天津", "山西", "广东",
                "江西", "广西", "新疆", "江苏",  "河北", "河南",
                "浙江", "海南", "湖北", "湖南", "甘肃", "福建",
                "西藏", "贵州", "辽宁", "重庆", "陕西", "青海", 
                "黑龙江","台湾", "香港", "澳门"]
provinceDict = {"云南":"YunNan", "内蒙古":"NeiMengGu", "北京":"BeiJing", "吉林":"JiLin",
                "四川":"SiChuan","上海":"ShangHai","宁夏":"NingXia", "安徽":"AnHui", 
                "山东":"ShanDong","天津":"TianJin", "山西":"ShanXi", "广东":"GuangDong",
                "江西":"JiangXi", "广西":"GuangXi", "新疆":"XinJiang", "江苏":"JiangSu",
                "河北":"HeBei", "河南":"HeNan","浙江":"ZheJiang", "海南":"HaiNan", 
                "湖北":"HuBei", "湖南":"HuNan", "甘肃":"GanSu", "福建":"FuJian",
                "西藏":"XiZang", "贵州":"GuiZhou", "辽宁":"LiaoNing", "重庆":"ChongQing", 
                "陕西":"ShanAnXi", "青海":"QingHai", "黑龙江":"HeiLongJiang","台湾":"TaiWan",
                "香港":"HongKong", "澳门":"Macao","南海诸岛":"South China Sea Islands"}
#drop useless columns
provinceName = ["YunNan", "NeiMengGu", "BeiJing", "JiLin",
                "SiChuan","ShangHai","NingXia", "AnHui", 
                "ShanDong","TianJin", "ShanXi", "GuangDong",
                "JiangXi", "GuangXi", "XinJiang", "JiangSu",
                "HeBei", "HeNan","ZheJiang", "HaiNan", 
                "HuBei", "HuNan", "GanSu", "FuJian",
                "XiZang", "GuiZhou", "LiaoNing", "ChongQing", 
                "ShanAnXi", "QingHai", "HeiLongJiang","TaiWan",
                "HongKong", "Macao"]


df.drop(['cityName', 'city_confirmedCount', 'city_suspectedCount',
         'city_curedCount', 'city_deadCount','continentName','continentEnglishName',
         'province_zipCode','cityEnglishName', 'city_zipCode' ], inplace=True, axis=1)

df['updateTime'] = pd.to_datetime(df['updateTime'])
df['updateTime'] = df['updateTime'].apply(lambda x: datetime.datetime.strftime(x, '%m-%d'))

In [75]:
#remove duplicate of updating information more than twice within one day
df = df.drop_duplicates(subset=['provinceName', 'updateTime'], keep='first',inplace=False)

In [76]:
Covid_df = df

In [77]:
Covid_df = Covid_df.replace(provinceDict)

In [78]:
indexOfForeign = Covid_df[Covid_df['countryName']!='中国'].index
Covid_df.drop(indexOfForeign, inplace=True)

In [79]:
Covid_df.shape

(1770, 10)

In [80]:
#remove duplicate that take China in the Province column
indexOfProv = Covid_df[Covid_df['provinceName']=='中国'].index
Covid_df.drop(indexOfProv, inplace=True)

In [81]:
#Covid_df

In [82]:
dateList = Covid_df['updateTime']

In [83]:
dateList.drop_duplicates(inplace=True)

In [84]:
dateList = dateList.to_list()
# dateList

In [85]:
Covid_df = Covid_df.dropna()

In [86]:
temptProvList = []
total_confirmed = []
total_suspect = []
total_dead = []
total_cured = []
date_used = []
confirmed_cases_dict = {}
suspected_dict = {}
dead_dict = {}
cured_dict = {}
ordered_date_list = dateList
ordered_date_list.reverse()
# ordered_date_list

In [87]:
def Reverse(lst):
    return [ele for ele in reversed(lst)]
time_list = Reverse(date_used)

In [88]:
for day in dateList:
    date_used.append(day)
    print(day)
    df = Covid_df[Covid_df['updateTime'] == day]
#     print(df)
    for idx in df.index:
        temptProvList.append(df.loc[idx, 'provinceName'])
    #find out the province missing data in one single day
    missingProvList = list( set(provinceName) - set(temptProvList))
    print(missingProvList)
    #find the num of confirmed suspected dead cured number of patients in previous day
    missing_prov_found = False
    prev_confirmed = 0
    prev_suspected = 0
    prev_cured = 0
    prev_dead = 0
    for prov in missingProvList:
#         print(prov)
        #return all days before current date
        back_up_list = ordered_date_list[:ordered_date_list.index(day)]
        back_up_list.reverse()
        for date in back_up_list:
#             print("missing date" + date)
            missing_prov_df = Covid_df[Covid_df['updateTime'] == date]
            for i in missing_prov_df.index:
                if str(missing_prov_df.loc[i,'provinceName']) == prov:
#                     print("true" + prov)
                    missing_prov_found = True
                    prev_confirmed = int(missing_prov_df.loc[i, 'province_confirmedCount'])
                    prev_suspected = int(missing_prov_df.loc[i, 'province_suspectedCount'])
                    prev_cured = int(missing_prov_df.loc[i, 'province_curedCount'])
                    prev_dead = int(missing_prov_df.loc[i, 'province_deadCount'])
#                     print(prov + "confirmed" + str(prev_confirmed))
                    break
            if missing_prov_found:
                missing_prov_found = False
#                 print("added")
#                 pd.DataFrame({"provinceName":prov, 
#                                     "province_confirmedCount":prev_confirmed,
#                                     "province_suspectedCount":prev_suspected,
#                                     "province_curedCount":prev_cured,
#                                     "province_deadCount":prev_dead })
                #add previous data to current date for missing prov
                Covid_df = Covid_df.append({"provinceName":prov, 
                                    "province_confirmedCount":prev_confirmed,
                                    "province_suspectedCount":prev_suspected,
                                    "province_curedCount":prev_cured,
                                    "province_deadCount":prev_dead,
                                    "updateTime":day},ignore_index = True)
#                 print("added")
                #bust break here or there would be multiple added value for missing province
                break
    temptProvList.clear()
    confirmed_cases_dict[day] = df['province_confirmedCount'].sum()
    suspected_dict[day] = df['province_suspectedCount'].sum()
    dead_dict[day] = df['province_deadCount'].sum()
    cured_dict[day] = df['province_curedCount'].sum()
    
    total_confirmed.append(confirmed_cases_dict[day])
    total_cured.append(cured_dict[day])
    total_dead.append(dead_dict[day])

01-22
['JiangSu', 'QingHai', 'ShanAnXi', 'NeiMengGu', 'HuBei', 'XiZang', 'GanSu', 'XinJiang']
01-23
['HeBei', 'QingHai', 'TaiWan', 'HuBei', 'HeNan', 'XiZang', 'ShanXi', 'TianJin', 'NingXia', 'YunNan']
01-24
['XiZang']
01-25
['Macao', 'XiZang']
01-26
['HongKong', 'QingHai', 'XiZang', 'JiLin']
01-27
['XiZang']
01-28
['HongKong', 'QingHai', 'XiZang']
01-29
['TaiWan', 'Macao', 'QingHai']
01-30
['HongKong', 'Macao']
01-31
['Macao', 'XiZang']
02-01
['TaiWan', 'Macao', 'GuiZhou', 'XiZang']
02-02
['TaiWan']
02-03
['TaiWan', 'Macao', 'XiZang']
02-04
[]
02-05
['TaiWan', 'Macao', 'XiZang', 'NingXia']
02-06
['XiZang']
02-07
['TaiWan', 'Macao', 'QingHai', 'XiZang']
02-08
['XiZang', 'Macao', 'QingHai']
02-09
['XiZang', 'Macao', 'QingHai']
02-10
['TaiWan', 'Macao', 'QingHai', 'XiZang']
02-11
['TaiWan', 'Macao', 'NeiMengGu', 'XiZang']
02-12
['TaiWan']
02-13
[]
02-14
['XiZang', 'GanSu', 'Macao', 'QingHai']
02-15
['TaiWan', 'Macao', 'XiZang', 'HongKong']
02-16
['XiZang', 'QingHai', 'NingXia']
02-17
['Xi

In [89]:
print(len(total_dead))

67


In [90]:
# confirmed_cases_dict

In [91]:
MapData = []
data = []

In [92]:
time_list.sort()
print(time_list)

[]


In [93]:
for day in dateList:
    df = Covid_df[Covid_df['updateTime'] == day]
    df['proportion'] = df['province_confirmedCount'] / confirmed_cases_dict[day]
    for idx in df.index:
        data.append({'name': df.loc[idx, 'provinceName'], 'value': [int(
            df.loc[idx, 'province_confirmedCount']), float(df.loc[idx, 'proportion']),
            str(df.loc[idx, 'provinceName']),df.loc[idx, 'province_suspectedCount'],
            df.loc[idx, 'province_curedCount'],df.loc[idx, 'province_deadCount']]})
    data = sorted(data, key=lambda x: -x['value'][1])
    MapData.append({'time': day, 'data': list(data)})
    data.clear()

/Users/zhangbowen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [94]:
#  print(MapData)


In [185]:
confirmed_list =[]
for num in total_confirmed:
    confirmed_list.append(int(num))

cured_List = []
for num in total_cured:
    cured_List.append(int(num))
    
dead_List = []
for num in total_dead:
    dead_List.append(int(num))
    
def render_chart(day:str):
    map_data = [
        [[x["name"], x["value"]] for x in d["data"]] for d in MapData if d["time"] == day
    ][0]
    
#   https://pyecharts.org/#/zh-cn/global_options?id=visualmapopts%ef%bc%9a%e8%a7%86%e8%a7%89%e6%98%a0%e5%b0%84%e9%85%8d%e7%bd%ae%e9%a1%b9
    map_chart = (
        Map()
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title = "China Covid-19 Tracking Project",
                subtitle = "How Covid-19 spread in China in more than two months",
                pos_left = 'center',
                pos_top="top",
                title_textstyle_opts = opts.TextStyleOpts(
                    font_size=30, color="#1A5276"
                    ),
            ),
#             visualmap_opts = opts.VisualMapOpts(
#                 is_calculable=True,
#                 dimension=0,
#                 pos_left="30",
#                 pos_top="center",
#                 range_text=["Severe", "Mild"],
#                 range_color=["#A3E4D7", "#F39C12", "#E74C3C"],
#                 textstyle_opts=opts.TextStyleOpts(color="#1A5276"),
#                 min_=0,
#                 max_=2000,
#             )
        )
        .add(
            series_name="",
            data_pair=map_data,
            name_map = provinceDict,
            zoom=0.9,
            center=[125, 35],
            is_map_symbol_show=True,
            itemstyle_opts={
                "normal": {"areaColor": "#D1F2EB", "borderColor": "#20252C"},
                "emphasis": {
                    "label": {"show": Timeline},
                    "areaColor": "rgba(255,255,255, 0.5)",
                },
            },
        )
    )
    
    line_chart = (
        Line()
        .add_xaxis(xaxis_data=ordered_date_list)
        .add_yaxis(series_name="confirmed",y_axis=confirmed_list,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.2),symbol='rect',symbol_size=5)
        .add_yaxis(series_name="cured",y_axis=cured_List,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.3),symbol='arrow',symbol_size=5)
        .add_yaxis(series_name="dead",y_axis=dead_List,
                   areastyle_opts=opts.AreaStyleOpts(opacity=1),symbol_size=5)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="Number of 'Confirmed' 'Cured' 'Dead' cases", pos_left="72%", pos_top="5%",
                title_textstyle_opts = opts.TextStyleOpts(font_size=15, color="#1A5276")
                
            ),
            visualmap_opts = opts.VisualMapOpts(
                    is_calculable=True,
                    dimension=0,
                    pos_left="30",
                    pos_top="center",
                    range_text=["Severe", "Mild"],
                    range_color=["#A3E4D7", "#F39C12", "#E74C3C"],
                    textstyle_opts=opts.TextStyleOpts(color="#1A5276"),
                    min_=0,
                    max_=2000,
            ),
            legend_opts = opts.LegendOpts(
                is_show = False
            )
        )
    )
    pie_data = [[x[0], x[1][0]] for x in map_data]
    pie = (
        Pie()
        .add(
            series_name="",
            data_pair=pie_data,
            radius=["10%", "25%"],
            center=["80%", "70%"],
            itemstyle_opts=opts.ItemStyleOpts(
                border_width=1, border_color="rgba(0,0,0,0.5)"
            ),
        )
        .set_global_opts(
            tooltip_opts=opts.TooltipOpts(is_show=True, formatter="{b} {d}%"),
            legend_opts=opts.LegendOpts(is_show=False),
            visualmap_opts = opts.VisualMapOpts(
                is_calculable=True,
                dimension=0,
                pos_left="30",
                pos_top="center",
                range_text=["Severe", "Mild"],
                range_color=["#85C1E9", "#F39C12", "#E74C3C"],
                textstyle_opts=opts.TextStyleOpts(color="#1A5276"),
                min_=0,
                max_=2000,
            )
        
        )
    )
#     sumsss = []
#     for num in total_confirmed:
#         sumsss.append(int(num))
#     line_chart = (
#         Line()
#         .add_xaxis(ordered_date_list)
#         .add_yaxis("", sumsss)
#     )
    grid_chart = (
        Grid()
        .add(line_chart,grid_opts=opts.GridOpts(pos_left="63%", pos_right="75", pos_top="12%", pos_bottom="66%") )
        .add(pie, grid_opts=opts.GridOpts(pos_left="80%", pos_top="15%"))
        .add(map_chart, grid_opts=opts.GridOpts())

    )

    return grid_chart

In [187]:
if __name__ == '__main__':
    timeline = Timeline(
        init_opts=opts.InitOpts(
            width="1800px", height="900px", theme="#7FB3D5")
    )
    for y in ordered_date_list:
        g = render_chart(day=y)
        timeline.add(g, time_point=str(y))

    
    timeline.add_schema(
        orient="horizontal",
        is_auto_play=True,
        is_inverse=False,
        play_interval=1200,
        pos_left="center",
        pos_right="null",
#         pos_top="20",
        pos_bottom="20",
        width="1500",
        label_opts=opts.LabelOpts(is_show=True, color="#1A5276"),
    )

    timeline.render("Covid-19-Vis.html")
#     print(MapData)